In [9]:
import pandas as pd
import json
from datetime import datetime
import requests
import os


In [10]:
with open('daten.json', 'r', encoding='utf-8') as datei:
    geladenes_dict = json.load(datei)

In [11]:
def finde_alle_ordner(daten):
    """
    Findet alle Ordner im Datawrapper-Konto.
    
    Args:
        daten: Die zu durchsuchende Datenstruktur
        
    Returns:
        Ein DataFrame mit Ordner-IDs und Namen
    """
    def durchsuche_rekursiv(daten):
        # Initialisiere folder_df am Anfang der Funktion
        folder_df = pd.DataFrame()
        
        if isinstance(daten, dict):
            # Prüfe, ob es sich um einen Ordner handelt
            if "id" in daten and "name" in daten:  # Hier habe ich "type" entfernt
                neuer_ordner = pd.DataFrame({
                    'id': [daten["id"]],
                    'name': [daten["name"]]
                })
                folder_df = pd.concat([folder_df, neuer_ordner], ignore_index=True)
            
            # Durchsuche alle Werte im Dict rekursiv
            for wert in daten.values():
                temp_df = durchsuche_rekursiv(wert)
                if not temp_df.empty:  # Nur konkatenieren wenn nicht leer
                    folder_df = pd.concat([folder_df, temp_df], ignore_index=True)
                    
        elif isinstance(daten, list):
            # Durchsuche alle Elemente in der Liste rekursiv
            for element in daten:
                temp_df = durchsuche_rekursiv(element)
                if not temp_df.empty:  # Nur konkatenieren wenn nicht leer
                    folder_df = pd.concat([folder_df, temp_df], ignore_index=True)
        
        return folder_df
    
    # Hier der tatsächliche Aufruf der rekursiven Funktion
    return durchsuche_rekursiv(daten)

In [12]:
df = finde_alle_ordner(geladenes_dict)


In [13]:
df

,id,name
0,295136,printexport
1,NcuSh8hB,RND
2,68040,sag
3,90909,Bundespräsident 2022
4,96563,Zeitumstellung
...,...,...
143,292842,Hanna
144,294004,printexport
145,297421,Christ
146,298024,BTW_2025


In [15]:
# Funktion zum Abrufen des Inhalts eines Datawrapper-Ordners anhand seiner ID
def hole_ordner_inhalt(ordner_id):
    """
    Ruft den Inhalt eines Datawrapper-Ordners über die API ab.
    
    Args:
        ordner_id: Die ID des Datawrapper-Ordners
        
    Returns:
        Ein DataFrame mit den Inhalten des Ordners
    """
    # API-Endpunkt für Ordnerinhalte
    url = f"https://api.datawrapper.de/v3/folders/{ordner_id}"
    
    API_TOKEN = os.getenv("DATAWRAPPER_API_KEY")
    
    # API-Anfrage senden
    response = requests.get(
        url,
        headers={"Authorization": f"Bearer {API_TOKEN}"}
    )
    
    # Prüfen, ob die Anfrage erfolgreich war
    if response.status_code == 200:
        # Daten aus der Antwort extrahieren
        daten = response.json()
        print(daten)
        
        # Wenn keine Daten vorhanden sind, leeren DataFrame zurückgeben
        if "data" not in daten or not daten["data"]:
            return pd.DataFrame()
        
        # Daten in einen DataFrame umwandeln
        items = []
        for item in daten["data"]:
            items.append({
                "id": item.get("id", ""),
                "type": item.get("type", ""),
                "title": item.get("title", ""),
                "lastModifiedAt": item.get("lastModifiedAt", ""),
                "createdAt": item.get("createdAt", "")
            })
        
        return pd.DataFrame(items)
    else:
        print(f"Fehler beim Abrufen des Ordnerinhalts: {response.status_code}")
        print(response.text)
        return pd.DataFrame()



df_ordner = hole_ordner_inhalt("NcuSh8hB")



Fehler beim Abrufen des Ordnerinhalts: 400
{"statusCode":400,"error":"Bad Request","message":"Invalid request payload input: \"id\" must be a number","type":"error / validation","details":[{"path":"id","type":"number.base"}]}


In [8]:
url = f"https://api.datawrapper.de/v3/folders/NcuSh8hB"
response = requests.get(
    url,
    headers={"Authorization": f"Bearer {api}"}
)
datastream = response.json()
print(datastream)


NameError: name 'api' is not defined

In [16]:
api = os.getenv("DATAWRAPPER_API_KEY")
print(api)


    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
# 
# ame aus dem datastream
df_folder = pd.DataFrame()
for id in df["id"]:
    url = f"https://api.datawrapper.de/v3/folders/{id}"
    response = requests.get(
        url,
        headers={"Authorization": f"Bearer {api}"}
    )
    datastream = response.json()
    print(datastream)
   


    if datastream:
        # Extrahiere die relevanten Informationen aus dem datastream
        folder_info = {
            'id': datastream.get('id', None),
            'name': datastream.get('name', None),
            'userId': datastream.get('userId', None),
            'parentId': datastream.get('parentId', None),
            'teamId': datastream.get('teamId', None), 
            "chartId": datastream.get('charts', None)
        }
        
        # Erstelle ein DataFrame mit den Folder-Informationen
        df_datastream = pd.DataFrame([folder_info])
        df_folder = pd.concat([df_folder, df_datastream], ignore_index=True)
        
print("DataFrame aus dem Ordner erstellt:")
print(df_folder)




z7FSi7hXXjc8fZa4L3bKNJJFBZwg1AwIGDYcc7wKueGfVr1p9mjVaG1X1NKPGlPd
{'id': 295136, 'name': 'printexport', 'userId': 839871, 'parentId': None, 'teamId': None, 'children': [], 'charts': [{'id': '9Uo9K'}]}
{'statusCode': 400, 'error': 'Bad Request', 'message': 'Invalid request payload input: "id" must be a number', 'type': 'error / validation', 'details': [{'path': 'id', 'type': 'number.base'}]}
{'id': 68040, 'name': 'sag', 'userId': None, 'parentId': None, 'teamId': 'NcuSh8hB', 'children': [{'id': 90909}, {'id': 96563}, {'id': 102620}, {'id': 121008}, {'id': 135671}, {'id': 142277}, {'id': 155667}, {'id': 159055}, {'id': 160962}, {'id': 162633}, {'id': 162911}, {'id': 163192}, {'id': 164867}, {'id': 171020}, {'id': 175384}, {'id': 191024}, {'id': 191025}, {'id': 204353}, {'id': 204931}, {'id': 223857}, {'id': 236852}, {'id': 246783}, {'id': 280771}, {'id': 282999}, {'id': 285266}, {'id': 288687}, {'id': 299259}, {'id': 299284}], 'charts': [{'id': '08RRA'}, {'id': '0JbI0'}, {'id': '0Q6Ch'}, 

In [23]:
response.json()

{'id': 292839,
 'name': 'Übung_Greta_Schulung',
 'userId': None,
 'parentId': None,
 'teamId': 'NcuSh8hB',
 'children': [{'id': 292841}, {'id': 292842}],
 'charts': [{'id': 'WrC63'}]}

In [1]:
df_folder

NameError: name 'df_folder' is not defined

In [10]:

# Suche nach allen Charts, die nach dem 1.4.2024 erstellt wurden
from datetime import datetime

def finde_charts_nach_datum(daten, datum_grenze):
    """
    Findet alle Charts, die nach einem bestimmten Datum erstellt wurden.
    
    Args:
        daten: Die zu durchsuchende Datenstruktur
        datum_grenze: Das Grenzdatum als datetime-Objekt
        
    Returns:
        Eine Liste mit chart_ids
    """
    chart_ids = []
    
    def durchsuche_rekursiv(daten):
        if isinstance(daten, dict):
            # Prüfe, ob es sich um einen Chart handelt
            if "id" in daten and "createdAt" in daten:
                try:
                    # Konvertiere das Erstellungsdatum in ein datetime-Objekt
                    created_date = datetime.fromisoformat(daten["createdAt"].replace('Z', '+00:00'))
                    
                    # Prüfe, ob das Chart nach dem Grenzdatum erstellt wurde
                    if created_date > datum_grenze:
                        chart_ids.append(daten["id"])
                except Exception as e:
                    print(f"Fehler beim Parsen des Datums für Chart {daten.get('id')}: {e}")
            
            # Durchsuche alle Werte im Dict rekursiv
            for wert in daten.values():
                durchsuche_rekursiv(wert)
                
        elif isinstance(daten, list):
            # Durchsuche alle Elemente in der Liste rekursiv
            for element in daten:
                durchsuche_rekursiv(element)
    
    durchsuche_rekursiv(daten)
    return chart_ids

# Setze das Grenzdatum auf den 1.4.2024
grenzdatum = datetime(2024, 4, 1, tzinfo=datetime.now().astimezone().tzinfo)

# Suche nach Charts, die nach dem 1.4.2024 erstellt wurden
neue_charts = finde_charts_nach_datum(geladenes_dict, grenzdatum)

# Ausgabe der Ergebnisse
print(f"Gefundene Charts nach dem 1.4.2024: {len(neue_charts)}")
print("\nListe der Chart-IDs:")
for chart_id in neue_charts:
    print(chart_id)

Gefundene Charts nach dem 1.4.2024: 2933

Liste der Chart-IDs:
9Uo9K
01moT
089Vk
0CWQ0
0FacC
0Ga38
0Hbdi
0Ip5v
0JUUR
0KqmN
0N6O8
0USsM
0Wued
0WuvR
0XF5f
0XKrt
0Z1Y9
0Z2Nf
0ZC8D
0aVnK
0c3IU
0gisZ
0irDM
0k9GQ
0lFsw
0mVhP
0qLGS
0sbd5
0wTcy
0yPMa
0ydUL
12M5D
12pjW
14QR9
17beO
1Axxu
1DPtL
1JcYi
1QDm9
1QsGg
1SE2L
1T095
1U0b4
1W0IV
1Wiyo
1XjEC
1Yosv
1aJtF
1as0Y
1kwXi
1nbtq
1pjA6
1tuIW
1uchl
1v57S
1vq7o
1wCKN
1xtPx
250Jw
265WK
2CnMA
2DiUy
2E96e
2NQty
2SRUi
2ZBKt
2dL1T
2gVz1
2kXM7
2uTL5
2vy5X
2xPA7
2ykuR
30kCy
32aJL
36cAb
3A2dE
3E0se
3Ey39
3Fieh
3FuBB
3Fze1
3H4UK
3KCYN
3LrCJ
3PGi1
3Q0XN
3RDdS
3TfoX
3UpgH
3VGZX
3W3Z6
3XBSi
3XHnX
3XlB7
3b7LE
3eXkw
3gUkJ
3iV4s
3ir79
3kTT5
3tUwN
3vpGA
3vts3
3xzyp
46JBK
4A315
4ApQx
4Ble5
4EO9b
4FfFG
4GeC6
4I1OY
4LSM5
4MaJW
4MsI6
4PRV7
4Q0Xr
4Qpon
4WWCT
4bsie
4c3mH
4iQcl
4uIbK
4xCqs
531Qu
559Kc
5Dg3i
5HFbm
5I8Ms
5Ih1u
5IpnV
5Ky8Y
5NNSe
5QIok
5RB6L
5UvZW
5VwMw
5XxRK
5adqf
5cL2f
5ekgD
5h8Kx
5hMnT
5mJxe
5nyYT
5oFJi
5uQ37
5wUre
5ywrN
60CSB
60wTg
61J9F
61iiE
61wiY
62FwM
6

In [12]:
len(neue_charts)

2933

In [ ]:
df = pd.read_csv("")